# A1367 OC - Image Reducer

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter

## Imports

In [ ]:
# Python Imports
from os import path
from glob import iglob

# Numerical Imports
import numpy as np

# Astropy Colab Imports
from astropy.io import fits
from astropy.wcs import WCS
from drizzlepac.tweakreg import TweakReg
from drizzlepac.tweakback import apply_tweak
from drizzlepac.astrodrizzle import AstroDrizzle
# import pyregion

## Notebook Setup

In [ ]:
# Data Directory
DATA_DIR = 'mastDownload/HST'

# FLC Glob Patter
FLC_CR_GLOB_PAT = path.join(DATA_DIR, '**/*_crclean_flc.fits')

# DRC Glob Patter
DRC_CR_GLOB_PAT = path.join(DATA_DIR, '**/*drc.fits')

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
flcNameDict = {}
for fn in iglob(FLC_CR_GLOB_PAT):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in flcNameDict:
        flcNameDict[filt] = []
    flcNameDict[filt].append(fn)

In [ ]:
flcNameDict

In [ ]:
# Get the File Names and Sort them by filter
drcNameDict = {}
for fn in iglob(DRC_CR_GLOB_PAT):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in drcNameDict:
        drcNameDict[filt] = []
    drcNameDict[filt].append(fn)

In [ ]:
drcNameDict

## Align DRCs to GAIA

### Align F475W DRC to GAIA

In [ ]:
# Align the Images to the GAIA data
TweakReg(
    drcNameDict['F475W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    catfile='catalogs/catfile',
    refcat='../Data/GAIA/A1367OC-GAIA-RefCatalog-icrs.txt',
    runfile='A1367OC-F475W-Tweak.log',
    searchrad=0.5,
    minobj=5,
    tolerance=3,
    interactive=False
)

### Align the F814W to F475W

In [ ]:
# DQ Bits
DQ_BAD_DET  = 4
DQ_HOT_PIX  = 16
DQ_CR_PIX   = 4096+8192
DQ_GOOD_PIX = ~(DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX)

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=10,
    conv_width=3.5,
)

# Align the Images to the GAIA data
TweakReg(
    drcNameDict['F814W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refimage=drcNameDict['F475W'][0],
    runfile='A1367OC-F814W-Tweak.log',
    searchrad=0.3,
    minobj=15,
    tolerance=1.00,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg,
    interactive=False
)

### Align the F275W to F475W

In [ ]:
# Image Find Parameters
imagefindcfg = dict(
    # peakmax=900,
    threshold=3,
    conv_width=3.5,
)
refimagefindcfg = dict(
    # peakmax=900,
    threshold=3,
    conv_width=3.5,
)

# Align the Images to the GAIA data
TweakReg(
    drcNameDict['F275W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refimage=drcNameDict['F475W'][0],
    # catfile='catalogs/catfile',
    runfile='A1367OC-F275W-Tweak.log',
    searchrad=0.5,
    minobj=5,
    tolerance=1,
    imagefindcfg=imagefindcfg,
    refimagefindcfg=refimagefindcfg,
    interactive=False
)

In [ ]:
%%bash
mkdir -p logs/tweak
mv *.log logs/tweak/
rm *.coo *.png

## TweakBack Solution

In [ ]:
# Loop through DRCs
for filter, fileList in drcNameDict.items():

    # Loop through the files
    for fn in fileList:

        # Get ASN
        asnName = fn.replace('drc.fits', 'asn.fits')

        # Get the Input Dither Files
        flcFiles = fits.getdata(asnName, 'ASN')
        flcFiles = flcFiles['MEMNAME'][flcFiles['MEMTYPE'] == 'EXP-DTH']
        flcFiles = [path.join(DATA_DIR, fn.lower(), f'{fn.lower()}_crclean_flc.fits') for fn in flcFiles]

        # Run Tweakback
        apply_tweak(
            fn + '[sci,1]', None, orig_wcs_key='A',
            input_files=','.join(flcFiles),
            tweaked_wcs_name='GAIA'
        )

## Drizzle Images for CR Correction

Although there will be additional notes added later, it is worth noting that according to
[STScI](https://hst-docs.stsci.edu/drizzpac/chapter-6-reprocessing-with-the-drizzlepac-package/6-3-running-astrodrizzle#id-6.3RunningAstroDrizzle-SelectingtheOptimalScaleandPixfrac):

1. For sub-pixel dithered data, select an output scale that's smaller than the native scale.
It will even help in the cosmic ray rejection step.
1. A smaller final_pixfrac gives higher resolution and lower correlated noise, but also reduces
sensitivity to low-surface brightness features (though it is possible to convolve a high resolution
image later to go after low surface brightness features).
1. Keep the standard deviation of the weight map over the main part of the image to above ~0.3 of
the mean to insure that one does not lose significant signal-to-noise in ignoring the weight map in
final photometry.

To summarize the last step, a `final_scale`/`final_pixfrac` combo should be chosen such that,
for the weight image,
\begin{equation}
    \mathrm{std} \gtrsim 0.3 \, \mathrm{mean}
\end{equation}

### Functions

In [ ]:
def check_weights(imageFileName, regionFileName):

    # Print the Findings
    def regions_stats(data):

        # Get Stats
        mean = np.nanmean(data)
        std = np.nanstd(data)
        thresh = 0.3*mean

        # Print the Stats
        print(f'Mean:   {mean:.3E}')
        print(f'Thresh: {thresh:.3E}')
        print(f'StdDev: {std:.3E}')
        print(std >= thresh, end='\n\n')

    # Get Regions
    regions = pyregion.open(regionFileName)

    # Open the Image
    with fits.open(imageFileName) as hduList:

        # Loop through Regions
        for region in regions:

            # Get the Mask
            mask = pyregion.ShapeList([region]).get_mask(hduList['WHT'])

            # Print Stats
            regions_stats(hduList['WHT'].data[mask])

### AstroDrizzle ACS Data Quality Flags

[Link to ACS/WFC DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)

In [ ]:
# DQ Bits
DQ_AS_CR    = 2
DQ_BAD_DET  = 4
DQ_STAB_HOT = 16
DQ_WARM_PIX = 64
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_BADREF   = 512
DQ_SINK_PIX = 1024
DQ_GOOD_PIX = DQ_WARM_PIX + DQ_FULL_WELL + DQ_SINK_PIX
# DQ_GOOD_PIX = ~4096

### Drizzle F814W Images

In [ ]:
from astropy.io import fits


# Drizzle Images
AstroDrizzle(
    flcNameDict['F814W'],
    output='A1367OC-F814W',
    runfile='F814W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='localmin',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.9,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_rot=0,
    final_scale=0.03
)


In [ ]:
check_weights('A1367OC-F814W_drc.fits', 'weight_regions/F814W.reg')

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    flcNameDict['F475W'],
    output='A1367OC-F475W',
    runfile='F475W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='localmin',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.9,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='A1367OC-F814W_drc.fits'
)

In [ ]:
check_weights('A1367OC-F475W_drc.fits', 'weight_regions/F475W.reg')

### AstroDrizzle WFC3 Data Quality Flags

[Link to ACS/WFC DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)

In [ ]:
# DQ Bits
DQ_AS_CR    = 2
DQ_BAD_DET  = 4
DQ_STAB_HOT = 16
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_BAD_FLAT = 512
DQ_SINK_PIX = 1024
DQ_GOOD_PIX = DQ_FULL_WELL + DQ_SINK_PIX
# DQ_GOOD_PIX = ~4096

### Drizzle F275W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    flcNameDict['F275W'],
    output='A1367OC-F275W',
    runfile='F275W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='localmin',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.33,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='A1367OC-F814W_drc.fits'
)

In [ ]:
check_weights('A1367OC-F275W_drc.fits', 'weight_regions/F275W.reg')

In [ ]:
%%bash
# Clean Up
mkdir -p logs/astro
mkdir -p ProcessedImages/HST/DrizzledImages
mv *Astro.log logs/astro/
mv *drc.fits ProcessedImages/HST/DrizzledImages/